```{contents}
```
## Thread Persistence

**Thread Persistence** in LangGraph is the mechanism that allows a workflow (graph execution) to **remember its state across time, process restarts, failures, and multiple user interactions**.
It transforms LangGraph from a simple pipeline executor into a **durable, long-running, stateful AI system**.

---

### **1. Core Intuition**

A LangGraph execution is identified by a **`thread_id`**.

> A **thread** = a persistent execution context
> **Thread persistence** = the ability to **pause, resume, and continue** that context at any time

Without thread persistence:
Each invocation is **stateless**.

With thread persistence:
The graph becomes a **durable process**.

---

### **2. Conceptual Model**

```
User Request ──► Thread ID ──► State Snapshot ──► Next Execution Step
                     ▲              │
                     └── Resume ◄───┘
```

The **entire graph state** is stored and reloaded on every invocation.

---

### **3. What Exactly Is Persisted**

| Component          | Description                       |
| ------------------ | --------------------------------- |
| State              | Shared typed state of the graph   |
| Execution Position | Which node is currently active    |
| History            | Messages, decisions, tool outputs |
| Pending Tasks      | In-flight operations              |
| Metadata           | Timestamps, user id, configs      |

This enables **crash-safe, resumable workflows**.

---

### **4. How Persistence Is Implemented**

LangGraph uses a **Checkpointer** abstraction.

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver("memory.db")
```

This object handles:

* Saving state after each node
* Restoring state before execution
* Versioning snapshots
* Managing threads

---

### **5. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict

class State(TypedDict):
    count: int

def increment(state):
    return {"count": state["count"] + 1}

builder = StateGraph(State)
builder.add_node("inc", increment)
builder.add_edge("inc", END)
builder.set_entry_point("inc")

graph = builder.compile(checkpointer=SqliteSaver("state.db"))

# First run
graph.invoke({"count": 0}, config={"thread_id": "user-1"})

# Second run continues same thread
graph.invoke({}, config={"thread_id": "user-1"})
```

The second call resumes from the previous state.

---

### **6. Lifecycle of a Persistent Thread**

| Phase      | Description                |
| ---------- | -------------------------- |
| Create     | New thread_id initialized  |
| Execute    | Nodes run and update state |
| Checkpoint | State is saved             |
| Pause      | Execution stops            |
| Resume     | Reload state and continue  |
| Finish     | Thread completes           |

---

### **7. Why Thread Persistence Matters**

| Problem                  | Without      | With                  |
| ------------------------ | ------------ | --------------------- |
| Multi-turn conversations | Lost context | Continuous memory     |
| Long workflows           | Must restart | Resume safely         |
| Failures                 | Work lost    | Recover automatically |
| Human review             | Hard         | Natural pause/resume  |
| Autonomous agents        | Impossible   | Reliable              |

---

### **8. Production Use Cases**

* Conversational assistants
* Approval workflows
* Research agents
* Monitoring systems
* Long-running automation
* Multi-user AI systems

---

### **9. Advanced Capabilities Enabled**

| Capability           | Enabled by Persistence |
| -------------------- | ---------------------- |
| Human-in-the-loop    | Yes                    |
| Autonomous loops     | Yes                    |
| Fault tolerance      | Yes                    |
| Scalability          | Yes                    |
| Audit & compliance   | Yes                    |
| System observability | Yes                    |

---

### **10. Comparison to Traditional Systems**

| Traditional Software | LangGraph Threads    |
| -------------------- | -------------------- |
| Stateless requests   | Stateful execution   |
| Sessions             | Persistent threads   |
| Transactions         | Durable workflows    |
| Jobs                 | Autonomous processes |

---

### **11. Mental Model**

Think of a LangGraph thread as:

> **A durable, resumable program execution that never forgets its state.**

### Demonstration

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver

# -------------------------
# State Definition
# -------------------------
class State(TypedDict):
    step: int
    message: str

# -------------------------
# Nodes
# -------------------------
def start(state: State):
    print("Running step:", state["step"])
    return {"step": state["step"] + 1}

def decide(state: State):
    if state["step"] >= 3:
        return {"message": "Process complete"}
    return {"message": "Continuing"}

# -------------------------
# Graph Construction
# -------------------------
builder = StateGraph(State)

builder.add_node("start", start)
builder.add_node("decide", decide)

builder.set_entry_point("start")
builder.add_edge("start", "decide")

builder.add_conditional_edges(
    "decide",
    lambda s: END if s["step"] >= 3 else "start",
    {"start": "start", END: END}
)

# -------------------------
# Enable Persistence
# -------------------------
checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


In [4]:
graph.invoke(
    {"step": 0, "message": ""},
    config={"thread_id": "job-42"}
)


Running step: 0
Running step: 1
Running step: 2


{'step': 3, 'message': 'Process complete'}